In [1]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [2]:
%cd /content/drive/MyDrive/Agent/

/content/drive/MyDrive/Agent


In [ ]:
!pip install qdrant-client

In [ ]:
!pip install sentence-transformers

In [ ]:
!pip install streamlit

In [14]:
import utils

In [15]:
from utils import get_earnings_transcript

quarter = "Q1"
ticker = "AAPL"
year = 2024
# resp_dict, speakers_list = get_earnings_transcript(quarter, ticker, year)
resp_dict, speakers_list = get_earnings_transcript(quarter, ticker, year)

In [16]:
resp_dict

{'symbol': 'AAPL',
 'quarter': 1,
 'year': 2024,
 'date': '2024-02-01 20:17:02',
 'content': "Operator: Good day, and welcome to the Apple Q1 Fiscal Year 2024 Earnings Conference Call. Today's call is being recorded. At this time, for opening remarks and introductions. I would like to turn the call over to Suhasini Chandramouli, Director of Investor Relations. Please go ahead.\nSuhasini Chandramouli: Thank you for joining us. Speaking first today is Apple's CEO, Tim Cook; and he'll be followed by CFO, Luca Maestri. After that, we'll open the call to questions from analysts. Before turning the call over to Tim, I would like to remind everyone that the quarter we're reporting today included 13 weeks, whereas the quarter we reported a year ago included 14 weeks. Please note that some of the information you'll hear during our discussion today will consist of forward-looking statements, including, without limitation, those regarding revenue, gross margin, operating expenses, other income an

In [17]:
speakers_list

['Krish Sankar',
 'Suhasini Chandramouli',
 'Luca Maestri',
 'Wamsi Mohan',
 'Ben Reitzes',
 'Mike Ng',
 'Amit Daryanani',
 'Erik Woodring',
 'Aaron Rakers',
 'Tim Cook',
 'Operator',
 'David Vogt']

In [18]:
print(resp_dict['content'])

Operator: Good day, and welcome to the Apple Q1 Fiscal Year 2024 Earnings Conference Call. Today's call is being recorded. At this time, for opening remarks and introductions. I would like to turn the call over to Suhasini Chandramouli, Director of Investor Relations. Please go ahead.
Suhasini Chandramouli: Thank you for joining us. Speaking first today is Apple's CEO, Tim Cook; and he'll be followed by CFO, Luca Maestri. After that, we'll open the call to questions from analysts. Before turning the call over to Tim, I would like to remind everyone that the quarter we're reporting today included 13 weeks, whereas the quarter we reported a year ago included 14 weeks. Please note that some of the information you'll hear during our discussion today will consist of forward-looking statements, including, without limitation, those regarding revenue, gross margin, operating expenses, other income and expense, taxes, capital allocation and future business outlook, including the potential impac

In [19]:
content = resp_dict['content']

In [20]:
import re

pattern = re.compile(r"\n(.*?):")
matches = pattern.finditer(content)

speakers_list = []
ranges = []
for match_ in matches:
    # print(match.span())
    span_range = match_.span()
    # first_idx = span_range[0]
    # last_idx = span_range[1]
    ranges.append(span_range)
    speakers_list.append(match_.group())

In [21]:
import re
speakers_list = [re.sub("\n","",sl) for sl in speakers_list]
speakers_list = [re.sub(":","",sl) for sl in speakers_list]

In [ ]:
!pip install langchain

In [23]:
from langchain.schema import Document

docs = []

for idx,speaker in enumerate(speakers_list[:-1]):
    start_range = ranges[idx][1]
    end_range = ranges[idx+1][0]
    speaker_text =  content[start_range+1:end_range]
    # docs.append({
    #     "speaker_text":speaker_text,
    #     "speaker":speaker
    # })
    docs.append(
        Document(page_content=speaker_text,metadata={"speaker":speaker})
    )
# docs.append({
#     "speaker_text":content[ranges[-1][1]:],
#     "speaker":speakers_list[-1]
# })

docs.append(
        Document(page_content=content[ranges[-1][1]:],metadata={"speaker":speakers_list[-1]})
    )

In [24]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 512,
    chunk_overlap  = 64,
    length_function = len,
    # is_separator_regex = False,
)

In [25]:
len(docs)

62

In [26]:
split_docs = text_splitter.split_documents(docs)

In [27]:
len(split_docs)

133

In [28]:
split_docs_qdrant = [{"speaker_text":doc.page_content,"speaker":doc.metadata['speaker']} for doc in split_docs]

In [30]:
# Import client library
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance

# qdrant_client = QdrantClient("http://localhost:6333")
qdrant_client = QdrantClient(":memory:")

In [31]:
from sentence_transformers import SentenceTransformer
import numpy as np
import json
import pandas as pd
from tqdm.notebook import tqdm
import torch
from qdrant_client import models, QdrantClient

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
encoder = SentenceTransformer(
    "all-MiniLM-L6-v2", device=device
)  # or device="cpu" if you don't have a GPU
print('1')
qdrant_client.recreate_collection(
    collection_name="earning_calls",
    vectors_config=VectorParams(size=encoder.get_sentence_embedding_dimension(), distance=Distance.COSINE),
)

cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

1


True

In [32]:
qdrant_client.upload_records(
    collection_name="earning_calls",
    records=[
        models.Record(
            id=idx, vector=encoder.encode(doc["speaker_text"]).tolist(), payload=doc
        )
        for idx, doc in enumerate(split_docs_qdrant)
    ],
)

<ipython-input-32-b205eaec1538>:1: DeprecationWarning: `upload_records` is deprecated, use `upload_points` instead
  qdrant_client.upload_records(


In [33]:
query_list = [
    "What was the Quarterly revenue",
    "How did Apple's revenue and profits in Q1 compare to the same quarter last year?",
    "What were the gross margins this quarter, and how do they reflect on Apple's cost management?",
    "What is the a breakdown of the revenue across different product lines like iPhones, iPads, Macs, Services, etc.?",
    "What were the best-selling products this quarter, and what trends are driving sales?",
    "How has the adoption rate for the latest iPhone model compared to previous models?",
    "How have international markets contributed to Apple’s revenue this quarter, especially in emerging markets?",
    "What impact has the current economic climate had on sales and product pricing strategies?",
    "How much has Apple invested in R&D this quarter, and in which areas?",
    "What is Apple's guidance for the next quarter and the rest of the fiscal year?",
    "What changes have been caused by COVID?",
]

In [35]:
from qdrant_client.models import VectorParams, Distance, Field, FieldCondition, MatchAny, Filter, Match
Answers = {}
for query_question in query_list:
    hits = qdrant_client.search(
        collection_name="earning_calls",
        # query_vector=encoder.encode("What was the Quarterly revenue").tolist(),
        # query_vector=encoder.encode("What are the financial figures for this quarter?").tolist(),
        query_vector=encoder.encode(query_question).tolist(),
        limit=5,
        query_filter=models.Filter(
            must=[
                models.FieldCondition(
                    key="speaker",
                    match=models.MatchAny(
                        any=speakers_list,
                    ),
                )
            ]
        ),
        search_params=models.SearchParams(hnsw_ef=256,exact=True)
    )

    relevant_docs = []
    for hit in hits:
        relevant_docs.append(hit.payload)

    relevant_docs
    relevant_speaker_text = {k:"" for k in speakers_list}

    for rd in relevant_docs:
        relevant_speaker_text[rd['speaker']]+=rd['speaker_text'] + " "
    relevant_text = "\n".join(f"{speaker}: {text}" for speaker, text in
                              relevant_speaker_text.items() if text.strip())
    Answers[query_question] = relevant_text

In [75]:
# relevant_speaker_text = {k:"" for k in speakers_list}

# for rd in relevant_docs:
#     relevant_speaker_text[rd['speaker']]+=rd['speaker_text'] + " "

In [78]:
# relevant_text = ""
# for speaker,text in relevant_speaker_text.items():
#     relevant_text+=speaker + ": "
#     relevant_text+=text + "\n"
# relevant_text = "\n".join(f"{speaker}: {text}" for speaker, text in
#                           relevant_speaker_text.items() if text.strip())



In [37]:
for i in Answers:
    print(i)
    print(Answers[i])

What was the Quarterly revenue
Tim Cook: Thank you. Suhasini. Good afternoon, everyone, and thanks for joining the call. Today, Apple is reporting revenue of $119.6 billion for the December quarter, up 2% from a year ago despite having one less week in the quarter. EPS was $2.18, up 16% from a year ago and an all-time record. We achieved revenue records across more than two dozen countries and regions including all-time records in Europe and rest of Asia-Pacific. We also continue to see strong double-digit growth in many emerging markets with 
Luca Maestri: Thank you, Tim, and good afternoon, everyone. Revenue for the December quarter was $119.6 billion, up 2% from last year. During the December quarter a year ago, two unique factors affected our results. First, we had an additional week in the quarter. And second, we had COVID-related factory shutdowns that limited iPhone supply. We estimate that the net impact of these two factors resulted in a 2 percentage point headwind to our reve